In [2]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import os
import pickle


In [3]:
def get_all_file(idprov, band):
    path_prov = '/data/ksa/03_Sampling/data-wide/'+idprov+'/'
    all_files_ = glob(path_prov+'*.pkl')
    files_ = [s for s in all_files_ if band in s]
    return files_

def get_all_file2(idprov, band):
    path_prov = '/data/ksa/04_Data_Preprocessing/'+idprov+'/01_imputation/wide_data/'
    all_files_ = glob(path_prov+'*.pkl')
    files_ = [s for s in all_files_ if band in s]
    return files_
    
def combine_all_data(band, idprov):
    files = get_all_file(idprov, band)
    df_sample_all = pd.DataFrame()
    for i in files:
        with open(i, 'rb') as f:
            df_tmp = pickle.load(f)
        df_sample_all = pd.concat([df_sample_all, df_tmp])
    return df_sample_all

def combine_all_data2(band, idprov):
    files = get_all_file2(idprov, band)
    df_sample_all = pd.DataFrame()
    for i in files:
        with open(i, 'rb') as f:
            df_tmp = pickle.load(f)
        df_sample_all = pd.concat([df_sample_all, df_tmp])
    return df_sample_all

def split_data(df, cutoff_date):
    df['year'] = df['periode'].str[:4]
    df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['nth'].astype(str), format='%Y-%m')

    # Split the dataset into training and testing sets
    training_set = df[df['date'] <= cutoff_date]
    testing_set = df[df['date'] > cutoff_date]

    return training_set, testing_set

In [4]:
# combine_all_data2(band, idprov).isna().sum()

In [ ]:
band = 'VV'
idprov='32'
cutoff_date = '2023-09-30'
df_all = combine_all_data2(band, idprov)
df_train, df_test = split_data(df_all, '2023-09-30')

In [ ]:
print(f'Training set: {len(df_train)} records')
print(f'Testing set: {len(df_test)} records')
print('Testing set proportion: ', str(round(100*(len(df_test))/(len(df_train)+len(df_test)) ,2)))

In [ ]:
with open('/data/ksa/04_Data_Preprocessing/training-test/'+idprov+'/training_imputation_'+idprov+'_'+band+'.pkl', 'wb') as f:
    pickle.dump(df_train, f)

with open('/data/ksa/04_Data_Preprocessing/training-test/'+idprov+'/testing_imputation_'+idprov+'_'+band+'.pkl', 'wb') as f:
    pickle.dump(df_test, f)

### Variance Viltering

In [6]:
with open('/data/ksa/04_Data_Preprocessing/32/02_variance_filtering/variance_filtering.pkl', 'rb') as f:
    df_var = pickle.load(f)
df_var['tahun'] = df_var['tahun'].replace({'22': '2022', '23': '2023'})
df_var.head()

,periode,idsubsegment,mean_var,tahun,bulan,obs,p5,p10,p20,p30,less_q5,less_q10,less_q20,less_q30
0,20220125_20220205,320101006A1,0.131505,2022,1,2.0,0.169263,0.222436,0.308105,0.384747,True,True,True,True
1,20220218_20220301,320101006A1,1.194184,2022,2,2.0,0.169263,0.222436,0.308105,0.384747,False,False,False,False
2,20220326_20220406,320101006A1,1.074512,2022,3,3.0,0.140487,0.183997,0.251232,0.312480,False,False,False,False
3,20220419_20220430,320101006A1,0.458386,2022,4,4.0,0.170888,0.225119,0.309007,0.386003,False,False,False,False
4,20220525_20220605,320101006A1,0.798636,2022,5,4.0,0.170888,0.225119,0.309007,0.386003,False,False,False,False


In [9]:
df_var_5 = df_var.loc[df_var.less_q5 == True,["idsubsegment","tahun","bulan","less_q5"]]
df_var_10 = df_var.loc[df_var.less_q10 == True,["idsubsegment","tahun","bulan","less_q10"]]

print(df_var.shape)
print(df_var.shape)
print(df_var_10.shape)

(432243, 14)
(432243, 14)
(43229, 4)


In [76]:
# df_var_5[['bulan','tahun']].drop_duplicates().sort_values(by=['tahun','bulan'])

In [10]:
band = 'VH'
idprov='32'
cutoff_date = '2023-09-30'
df_all = combine_all_data2(band, idprov)
df_all['year'] = df_all['periode'].str[:4]
df_all['nth'] = df_all['nth'].astype("str")

In [11]:
print(df_all.shape)
df_all.head()

(12287100, 40)


,idpoint,idsubsegment,idsegment,nth,periode,observation,class,MGRS,VH_30,VH_29,...,VH_8,VH_7,VH_6,VH_5,VH_4,VH_3,VH_2,VH_1,VH_0,year
0,320201009A1#01,320201009A1,320201009,1,2022_03,7.0,NV,48MXS,-13.950575,-13.704084,...,-14.433197,-13.482984,-12.038650,-12.308728,-12.026812,-12.877756,-14.129539,-13.121758,-13.279804,2022
1,320201009A1#02,320201009A1,320201009,1,2022_03,7.0,NV,48MXS,-12.331718,-11.972934,...,-12.922941,-12.722636,-12.356975,-11.897697,-11.334288,-12.226791,-13.437385,-12.356170,-13.047971,2022
2,320201009A1#03,320201009A1,320201009,1,2022_03,7.0,NV,48MXS,-11.529987,-11.416285,...,-13.252157,-13.417534,-13.683514,-13.061878,-12.575087,-12.567230,-12.675266,-12.290981,-13.212626,2022
3,320201009A1#04,320201009A1,320201009,1,2022_03,7.0,NV,48MXS,-13.361443,-13.600700,...,-13.288183,-14.324576,-15.200937,-13.972915,-12.964134,-12.838857,-12.582704,-12.307726,-12.659531,2022
4,320201009A1#05,320201009A1,320201009,1,2022_03,7.0,NV,48MXS,-12.995016,-13.444426,...,-13.607082,-14.734580,-15.240816,-14.630651,-14.336236,-13.393566,-12.625300,-12.127337,-12.909364,2022


In [12]:
df_merge = df_all.merge(df_var_10, how='left', left_on = ['year','nth','idsubsegment'], right_on=['tahun','bulan','idsubsegment'])
df_merge = df_merge[df_merge['less_q10'].notnull()]

print(df_merge.shape)
df_merge.head()

(1076875, 43)


,idpoint,idsubsegment,idsegment,nth,periode,observation,class,MGRS,VH_30,VH_29,...,VH_5,VH_4,VH_3,VH_2,VH_1,VH_0,year,tahun,bulan,less_q10
25,320201009A2#01,320201009A2,320201009,1,2022_03,2.0,V2,48MXS,-13.300971,-14.100189,...,-13.173245,-14.038830,-13.904123,-14.517735,-12.964956,-13.357330,2022,2022,1,True
26,320201009A2#02,320201009A2,320201009,1,2022_03,2.0,V2,48MXS,-14.137568,-14.138426,...,-12.656797,-14.190660,-13.979087,-14.409728,-13.998713,-14.271786,2022,2022,1,True
27,320201009A2#03,320201009A2,320201009,1,2022_03,2.0,V2,48MXS,-14.327194,-13.893816,...,-15.470042,-16.777348,-16.501491,-16.768417,-15.419333,-15.126764,2022,2022,1,True
28,320201009A2#04,320201009A2,320201009,1,2022_03,2.0,V2,48MXS,-14.361313,-13.703333,...,-15.880199,-16.604057,-16.364377,-16.118856,-16.537674,-15.598545,2022,2022,1,True
29,320201009A2#05,320201009A2,320201009,1,2022_03,2.0,V2,48MXS,-15.131207,-14.825255,...,-15.453243,-15.345831,-15.700595,-15.503248,-17.343239,-15.933994,2022,2022,1,True


In [ ]:
# df_train.groupby(['year','nth']).size()

In [13]:
df_train, df_test = split_data(df_merge, '2023-09-30')

In [14]:
print(f'Training set: {len(df_train)} records')
print(f'Testing set: {len(df_test)} records')
print('Testing set proportion: ', str(round(100*(len(df_test))/(len(df_train)+len(df_test)) ,2)))

Training set: 1076875 records
Testing set: 0 records
Testing set proportion:  0.0


In [15]:
with open('/data/ksa/04_Data_Preprocessing/training-test/'+idprov+'/training_imputation_varfilter10_'+idprov+'_'+band+'.pkl', 'wb') as f:
    pickle.dump(df_train, f)

# with open('/data/ksa/04_Data_Preprocessing/training-test/'+idprov+'/testing_imputation_varfilter_'+idprov+'_'+band+'.pkl', 'wb') as f:
#     pickle.dump(df_test, f)

## cek

In [4]:
'/data/ksa/04_Data_Preprocessing/training-test/'+idprov+'/training_imputation_'+idprov+'_'+band+'.pkl'

'/data/ksa/04_Data_Preprocessing/training-test/32/training_imputation_32_VV.pkl'

In [7]:
band = 'VV'
idprov='32'
with open('/data/ksa/04_Data_Preprocessing/training-test/'+idprov+'/training_imputation_'+idprov+'_'+band+'.pkl', 'rb') as f:
    df = pickle.load(f)

In [8]:
df.groupby('observation').size()

observation
0.0    1242600
1.0    1683475
2.0    1302225
3.0    1846325
4.0    1263575
5.0     760750
6.0    1381675
7.0    1263550
dtype: int64

In [19]:
df_train.isna().sum()


idpoint             0
idsubsegment        0
idsegment           0
nth                 0
periode             0
observation     23375
class           23375
MGRS                0
VV_30               0
VV_29               0
VV_28               0
VV_27               0
VV_26               0
VV_25               0
VV_24               0
VV_23               0
VV_22               0
VV_21               0
VV_20               0
VV_19               0
VV_18               0
VV_17               0
VV_16               0
VV_15               0
VV_14               0
VV_13               0
VV_12               0
VV_11               0
VV_10               0
VV_9                0
VV_8                0
VV_7                0
VV_6                0
VV_5                0
VV_4                0
VV_3                0
VV_2                0
VV_1                0
VV_0                0
year                0
date                0
dtype: int64